In [109]:
import pandas as pd
import os
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas_ml as pdml
from imblearn.over_sampling import SMOTE
import random

###  Data Preparation

In [110]:
data= pd.read_csv(r"C:\Users\DELL\Desktop\STAT Project\creditcard.csv")  #reading data
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [111]:
data["Amount_n"] = StandardScaler().fit_transform(data['Amount'].reshape(-1, 1))  #Normalizing Amount & replacing it in Amount_n
data=data.drop(columns=['Amount','Time'])                                         #removing Time and Amount

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [112]:
data.tail()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,Amount_n
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0,-0.350151
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,0,-0.254117
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,0,-0.081839
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,0,-0.313249
284806,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,-0.915427,...,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,0,0.514355


#### Generating balanced data using SMOTE 

In [113]:
df= pdml.ModelFrame(data, target='Class')            #the variable class will now be referred to as Target
df.target.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [114]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,Amount_n
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073403


In [115]:
sampler = df.imbalance.over_sampling.SMOTE()

In [116]:
sampler

SMOTE(k=None, k_neighbors=5, kind='regular', m=None, m_neighbors=10, n_jobs=1,
   out_step=0.5, random_state=None, ratio='auto', svm_estimator=None)

In [117]:
sampled=df.fit_sample(sampler)
sampled.target.value_counts()

1    284315
0    284315
Name: Class, dtype: int64

In [121]:
sampled=pd.DataFrame(data=sampled)


In [127]:
neg_cl=sampled.loc[sampled['Class']==0]
neg_cl=pd.DataFrame(data=neg_cl)

pos_cl=sampled.loc[sampled['Class']==1]
pos_cl=pd.DataFrame(data=pos_cl)


In [128]:
#Reducing the dataset to 50000 samples so that it is easier to compute
neg_cl=neg_cl.sample(n=25000)
pos_cl=pos_cl.sample(n=25000)

finalDf = neg_cl.append(pos_cl, ignore_index=True)

In [143]:
X = finalDf.iloc[:,1:30] 
y = finalDf.iloc[:,0] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0)  #creating test and train sets

### Fitting data using Logistic regression model

In [144]:
#importig module
from sklearn.linear_model import LogisticRegression
#making the instance
classifier = LogisticRegression(random_state=0)
#learning
classifier.fit(X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [145]:
#Prediction
prediction=classifier.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))

Accuracy: 0.944466666667
Confusion Metrix:
 [[7361  636]
 [ 197 6806]]


In [146]:
from sklearn.metrics import classification_report
print(classification_report(prediction, y_test))

             precision    recall  f1-score   support

          0       0.97      0.92      0.95      7997
          1       0.91      0.97      0.94      7003

avg / total       0.95      0.94      0.94     15000



### SVM

In [147]:

#SVM
#importing module
from sklearn import svm
#making the instance
model = svm.SVC(kernel='linear',random_state=0)
#learning
model.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [148]:
#Prediction
prediction=model.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))

Accuracy: 0.943333333333
Confusion Metrix:
 [[7374  666]
 [ 184 6776]]


In [149]:
from sklearn.metrics import classification_report
print(classification_report(prediction, y_test))

             precision    recall  f1-score   support

          0       0.98      0.92      0.95      8040
          1       0.91      0.97      0.94      6960

avg / total       0.95      0.94      0.94     15000



### Random Forest

In [150]:
from sklearn.ensemble import RandomForestClassifier
#making the instance
model=RandomForestClassifier(n_jobs=-1,random_state=123)
#learning
model.fit(X_train,y_train)
#Prediction
prediction=model.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))

Accuracy: 0.994866666667
Confusion Metrix:
 [[7541   60]
 [  17 7382]]


In [153]:
from sklearn.metrics import classification_report
print(classification_report(prediction, y_test))

             precision    recall  f1-score   support

          0       1.00      0.99      0.99      7601
          1       0.99      1.00      0.99      7399

avg / total       0.99      0.99      0.99     15000



### kNN

In [154]:
from sklearn.neighbors import KNeighborsClassifier
#making the instance
model = KNeighborsClassifier(n_jobs=-1)
#learning
model.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

In [156]:
#Prediction
prediction=model.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))

Accuracy: 0.992333333333
Confusion Metrix:
 [[7443    0]
 [ 115 7442]]


In [157]:
from sklearn.metrics import classification_report
print(classification_report(prediction, y_test))

             precision    recall  f1-score   support

          0       0.98      1.00      0.99      7443
          1       1.00      0.98      0.99      7557

avg / total       0.99      0.99      0.99     15000



# Fitting Data with Hypermetric Tuning

### Logistic Regression

In [158]:
# Import necessary modules
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Setup the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space}

# Instantiate a logistic regression classifier: logreg
logreg = LogisticRegression()

# Instantiate the GridSearchCV object: logreg_cv
logreg_cv = GridSearchCV(logreg, param_grid, n_jobs=-1)

# Fit it to the data
logreg_cv.fit(X_train, y_train)

# Print the tuned parameter and score
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_))
print("Best score is {}".format(logreg_cv.best_score_))

Tuned Logistic Regression Parameters: {'C': 31.622776601683793}
Best score is 0.9438285714285715


In [159]:
#Prediction
prediction=logreg_cv.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))

Accuracy: 0.944733333333
Confusion Metrix:
 [[7362  633]
 [ 196 6809]]


In [160]:
from sklearn.metrics import classification_report
print(classification_report(prediction, y_test))

             precision    recall  f1-score   support

          0       0.97      0.92      0.95      7995
          1       0.91      0.97      0.94      7005

avg / total       0.95      0.94      0.94     15000



### SVM

In [161]:
#With Hyper Parameters Tuning
#2-3,SVM
#importing modules
from sklearn.model_selection import GridSearchCV
from sklearn import svm
#making the instance
model=svm.SVC()
#Hyper Parameters Set
params = {'C': [6,7,8,9,10,11,12], 
          'kernel': ['linear','rbf']}
#Making models with hyper parameters sets
model1 = GridSearchCV(model, param_grid=params, n_jobs=-1)
#Learning
model1.fit(X_train,y_train)
#The best hyper parameters set
print("Best Hyper Parameters:\n",model1.best_params_)

Best Hyper Parameters:
 {'C': 12, 'kernel': 'rbf'}


In [162]:
#Prediction
prediction=model1.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))

Accuracy: 0.9982
Confusion Metrix:
 [[7531    0]
 [  27 7442]]


In [163]:
from sklearn.metrics import classification_report
print(classification_report(prediction, y_test))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7531
          1       1.00      1.00      1.00      7469

avg / total       1.00      1.00      1.00     15000



### Random Forest

In [164]:
#With Hyper Parameters Tuning
#2-2,Randomforest
#importing modules
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
#making the instance
model=RandomForestClassifier()
#hyper parameters set
params = {'criterion':['gini','entropy'],
          'n_estimators':[10,15,20,25,30],
          'min_samples_leaf':[1,2,3],
          'min_samples_split':[3,4,5,6,7], 
          'random_state':[123],
          'n_jobs':[-1]}
#Making models with hyper parameters sets
model1 = GridSearchCV(model, param_grid=params, n_jobs=-1)
#learning
model1.fit(X_train,y_train)
#The best hyper parameters set
print("Best Hyper Parameters:\n",model1.best_params_)

Best Hyper Parameters:
 {'criterion': 'entropy', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 30, 'n_jobs': -1, 'random_state': 123}


In [165]:
#Prediction
prediction=model1.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))

Accuracy: 0.998133333333
Confusion Metrix:
 [[7544   14]
 [  14 7428]]


In [166]:
from sklearn.metrics import classification_report
print(classification_report(prediction, y_test))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7558
          1       1.00      1.00      1.00      7442

avg / total       1.00      1.00      1.00     15000



### kNN

In [167]:
#With Hyper Parameters Tuning
#2-4,kNearestNeighbors
#importing modules
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
#making the instance
model = KNeighborsClassifier(n_jobs=-1)
#Hyper Parameters Set
params = {'n_neighbors':[5,6,7,8,9,10],
          'leaf_size':[1,2,3,5],
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree'],
          'n_jobs':[-1]}
#Making models with hyper parameters sets
model1 = GridSearchCV(model, param_grid=params, n_jobs=1)
#Learning
model1.fit(X_train,y_train)
#The best hyper parameters set
print("Best Hyper Parameters:\n",model1.best_params_)


Best Hyper Parameters:
 {'algorithm': 'auto', 'leaf_size': 1, 'n_jobs': -1, 'n_neighbors': 6, 'weights': 'distance'}


In [169]:
#Prediction
prediction=model1.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))

Accuracy: 0.9928
Confusion Metrix:
 [[7450    0]
 [ 108 7442]]


In [170]:
from sklearn.metrics import classification_report
print(classification_report(prediction, y_test))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      7450
          1       1.00      0.99      0.99      7550

avg / total       0.99      0.99      0.99     15000

